In [4]:
# Imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

# Headers for website requests
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) Gecko/20100101 Firefox/120.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",  # Appears as if you're coming from Google
    "Connection": "keep-alive",
    "DNT": "1",  # Do Not Track request
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
}

class Website:

    def __init__(self, url):
        """
        Initialize the website object and fetch content.
        """
        self.url = url
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an error for bad responses
        except requests.RequestException as e:
            print(f'Failed to fetch website: {e}')
            self.title = 'Error fetching website'
            self.text = ''
            return
        
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = 'No readable content found'

url = input('Write down your website URL here: \n')
website = Website(url)

# Step 1: Create prompts

system_prompt = (
    "You are an senior research scientist that analyzes the contents of a website \
    and provides a short summary, ignoring text that might be navigation related. \
    Respond in markdown."
)
user_prompt = (
    f"You are looking at a website titled {website.title}. \n\
    The contents of this website is as follows; \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n\
    {website.text}"
)

# Step 2: Construct messages list
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step3: Prepare API Request Payload
payload = {
    'model': MODEL,
    'messages': messages,
    'stream': False
}

# Step 4: Call Llama 3.2 and Handle Response
try:
    response = requests.post(OLLAMA_API, 
                             json=payload, 
                             headers=HEADERS)
    result = response.json().get('message', {}).get('content', 'No response received.')
    print(result)
except Exception as e:
    print(f'Llama API call failed: {e}')

Write down your website URL here: 
https://www.nlab-utsw.org
**Summary of N-LAB Website**

The N-LAB website is dedicated to developing novel neuroimaging and neuroengineering methods to integrate molecular and system neuroscience. The lab aims to understand the neural mechanisms of reward, learning, and memory in rodents' brains.

**Key Information**
-------------------

* The N-LAB's mission is to develop innovative approaches for brain science research.
* They are located at UT Southwestern Medical Center and have access to state-of-the-art imaging resources.
* The lab has strong support from the Advanced Imaging Research Center (AIRC) and can utilize various core facilities on campus.

**News/Announcements**
----------------------

None found. However, the website appears to be using Google Sites, which may indicate that updates or announcements are being made through this platform.
